In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2 as pg
from io import StringIO
from datetime import datetime
from enum import Enum
import time
import requests
%matplotlib inline

In [8]:
connection = pg.connect(user="admin",
                            password="quest",
                            host="127.0.0.1",
                            port="8812",
                            database="qdb",
                            options='-c statement_timeout=300000')

In [9]:
DEVICE_TO_DB_SENSOR_NAME = {

    'accelerometeruncalibrated': 'acc',
    'gyroscopeuncalibrated': 'gyro',
    'magnetometeruncalibrated': 'mag'

}

In [10]:
query = f"""select * from device_offload  """

pd.read_sql(query, connection)

/var/folders/9r/y3yrm7yx2x31dxvgqt8pww3h0000gn/T/ipykernel_52915/423303796.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, connection)


,device_id,session_id,device_timestamp,recorded_timestamp,sensor_name,x,y,z
0,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,90491c82-0f7c-4558-b649-e21540203c7e,2022-12-05 01:29:48.886707,2022-12-05 01:29:49.427948,mag,3340.550,532.625,-5060.119
1,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,90491c82-0f7c-4558-b649-e21540203c7e,2022-12-05 01:29:48.884146,2022-12-05 01:29:49.427969,acc,0.027,-0.445,-0.914
2,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,90491c82-0f7c-4558-b649-e21540203c7e,2022-12-05 01:29:48.889160,2022-12-05 01:29:49.427984,gyro,-0.101,-0.124,-0.018
3,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,90491c82-0f7c-4558-b649-e21540203c7e,2022-12-05 01:29:48.906728,2022-12-05 01:29:49.428000,mag,3341.289,533.121,-5060.145
4,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,90491c82-0f7c-4558-b649-e21540203c7e,2022-12-05 01:29:48.904201,2022-12-05 01:29:49.428064,acc,0.014,-0.441,-0.880
...,...,...,...,...,...,...,...,...
9445,18eeccac-30c7-4735-a967-102b140a3138,585fc84d-5a6c-4ec3-96c9-8d5b0c76f005,2022-12-05 01:33:34.308069,2022-12-05 01:33:34.350767,acc,0.114,-0.303,-0.882
9446,18eeccac-30c7-4735-a967-102b140a3138,585fc84d-5a6c-4ec3-96c9-8d5b0c76f005,2022-12-05 01:33:34.318038,2022-12-05 01:33:34.350876,acc,0.086,-0.342,-0.825
9447,18eeccac-30c7-4735-a967-102b140a3138,585fc84d-5a6c-4ec3-96c9-8d5b0c76f005,2022-12-05 01:33:34.328006,2022-12-05 01:33:34.350884,acc,0.095,-0.354,-0.792
9448,18eeccac-30c7-4735-a967-102b140a3138,585fc84d-5a6c-4ec3-96c9-8d5b0c76f005,2022-12-05 01:33:34.337974,2022-12-05 01:33:34.350893,acc,0.117,-0.370,-0.821


In [4]:
def write_sensor_payloads(data:dict, table_name:str):
    
    """
    Write phone sensor data to database tables

    Parameters
    ----------
    data : dict
        The raw request data sent by the phone
    table_name : str
        The name of the table to write to 
    """

    session_id = data['sessionId']
    device_id = data['deviceId']

    # Create an empty dict to store structured sensor from the payload
    structured_payload = {'device_id':[],
                            'session_id':[],
                            'device_timestamp':[],
                            'recorded_timestamp':[],
                            'sensor_name':[],
                            'x':[],
                            'y':[],
                            'z':[]
                            }
    
    for d in data['payload']:

        # Triaxial sensors
        if d.get("name") in ['accelerometeruncalibrated','gyroscopeuncalibrated','magnetometeruncalibrated']:

            structured_payload['device_id'].append(device_id)
            structured_payload['session_id'].append(session_id)
            structured_payload['device_timestamp'].append(str(datetime.fromtimestamp(int(d["time"]) / 1000000000)))
            structured_payload['recorded_timestamp'].append(str(datetime.utcnow()))
            structured_payload['sensor_name'].append(DEVICE_TO_DB_SENSOR_NAME.get(d.get("name")))
            structured_payload['x'].append(d["values"]["x"])
            structured_payload['y'].append(d["values"]["y"])
            structured_payload['z'].append(d["values"]["z"])  

    output = StringIO()
    pd.DataFrame(structured_payload).to_csv(output, sep=',', header=True, index=False)
    output.seek(0)
    contents = output.getvalue()
    csv = {'data': ('device_offload', contents)}
    server = 'http://localhost:9000/imp'
    response = requests.post(server, files=csv)


In [6]:
data = pd.read_sql("select * from device_offload", connection)
data.sort_values('recorded_timestamp')

/var/folders/9r/y3yrm7yx2x31dxvgqt8pww3h0000gn/T/ipykernel_43662/2068274001.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql("select * from device_offload", connection)


,device_id,session_id,device_timestamp,recorded_timestamp,sensor_name,x,y,z
0,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:15:56.776696,acc,43.000,31.000,12.000
1,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:15:56.776707,gyro,43.000,31.000,12.000
2,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:15:56.776713,mag,43.000,31.000,12.000
3,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:16:49.918553,acc,43.000,31.000,12.000
4,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:16:49.918563,gyro,43.000,31.000,12.000
...,...,...,...,...,...,...,...,...
3092,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,f86413ab-30e5-4766-a8e0-0e7fa5e8490b,2022-11-30 16:06:04.451108,2022-11-30 16:06:04.491614,acc,0.211,-0.114,-0.977
3093,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,f86413ab-30e5-4766-a8e0-0e7fa5e8490b,2022-11-30 16:06:04.465624,2022-11-30 16:06:04.491625,mag,3440.150,594.465,-5061.349
3094,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,f86413ab-30e5-4766-a8e0-0e7fa5e8490b,2022-11-30 16:06:04.471163,2022-11-30 16:06:04.491635,gyro,0.018,-0.079,0.064
3095,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,f86413ab-30e5-4766-a8e0-0e7fa5e8490b,2022-11-30 16:06:04.471163,2022-11-30 16:06:04.491645,acc,0.218,-0.115,-0.985
